In [1]:
import os
import pandas as pd
import dask.dataframe as dd
from knowledge_graph_generator import KnowledgeGraphGenerator
from fusion import TransE, TransEFuser

import seaborn as sns
import matplotlib.pyplot as plt


In [24]:
dask_df = dd.read_csv("data/relation_files/*.csv")
full_df = dask_df.compute()

fuser = TransEFuser(n_epochs=1)
kg_df = fuser.fuse(full_df)

In [25]:
filtered_df = kg_df.loc[kg_df["probability"] > 0.75].reset_index(drop=True)
x = filtered_df[filtered_df["entity_id"] == "m/spiderman_2"]
for i in range(len(filtered_df[filtered_df["entity_id"] == "m/spiderman"])):
    r = x.iloc[i]
    print(r["relation"], r["value"], r["probability"])

features the miscellaneous theme post 0.8309313930342925
features the miscellaneous theme Phase Three 0.8770152381337388
features the miscellaneous theme Godzilla : King of the Monsters 0.7556307173199088
features the miscellaneous theme Phase 3 0.9637231179948752
features the miscellaneous theme Spider-man 0.8702799825193193
features the miscellaneous theme Iron Man 4 0.800544004393814
features the miscellaneous theme Dutch 0.8396888292442442
features the miscellaneous theme MCU Spidey 0.8386825553487078
features the miscellaneous theme Marvel Universe 0.8303054810285748
features the miscellaneous theme Home 0.7639676883375516
features the miscellaneous theme ey 0.8045822739126643
features the miscellaneous theme The Greatest American Hero : The Movie 0.8240666894523073
features the miscellaneous theme Spider Man : Homecoming 0.7853025169230101
features the miscellaneous theme Spider-Man : Into the Spider-Verse 0.8680619640618144
features the miscellaneous theme Spider-Men and Women 0